### Step3. Obtaining allele specific counts and predict ASCAVs

At this step, we aim to obtain allele specific counts. We first start by comparing HAP1 and HAP2 bam files generated in the first step.
We evaluate each read in these bam files in order to identify its most likely origin. We do this by looking first at mapping quality, then at the number of mismatches. If a read has higher mapping quality for either of the haplotypes it is assigned to that haplotype. If the mapping quality is the same, we look at the number of mismatches and assigned the read to the haplotype for which the read is aligned with least number of mismatches. If both metrics are equivalent, we deem the read as commonly mapping to both haplotypes. This step results in four bam files: HAP1.unique.bam, HAP2.unique.bam, HAP1.common.bam and HAP2.common.bam (HAP1.common.bam and HAP2.common.bam should essentially be identical).
We next check duplicate and ambiguously mapping reads. We mark the duplicate reads in each of these bam files with Picard. Then, by lifting over the read positions in HAP1.common.bam and HAP2.common.bam to reference genome, we check for ambiguously mapping reads (if the reads are truly indistinguishable between two haplotypes, they should correspond to the same location when lifted over to the reference genome).
Next, we obtain phased heterozygous variants from whole genome sequence data and overlap them with consolidated peaks obtained in the previous step. These variants will be the ones that we will try to obtain allele specific counts. SNV locations in peaks are lifted over to HAP1 and HAP2, and we count the alleles at these variants using HAP1.bam and HAP2.bam files with samtools mpileup. We also count the alleles using the COMMON.bam file.
Next, we combine allelic counts from HAP1, HAP2 and COMMON alignments, add genomic allelic ratios from WGS data, and predict allele specific chromatin accessibility variants using BaalCHIP.

In [ ]:
# load necessary modules
module load SAMtools/1.9-20181106-foss-2014a
module load bedtools/20181008-foss-2014a
module load Kent/20180816
module load Picard/2.18.11-Java-jdk1.8.0_151

In [ ]:
# create directories

In [ ]:
# give sample name here
sample="MM031"

In [ ]:
# create haplotype specific bam files
/staging/leuven/stg_00002/lcb/zkalender/software/src_zkalender/melanoma_WGS_project/postprocess_hap1_and_hap2_bam_files_v2.sh ${sample}

In [ ]:
# mark duplicates in unique and commonly mapping reads
picard MarkDuplicates \
    I=${sample}_HAP1.common.bam \
    O=${sample}_HAP1.common.dup_marked.bam \
    CREATE_INDEX=false \
    VALIDATION_STRINGENCY=SILENT \
    REMOVE_DUPLICATES=FALSE \
    M=${sample}_HAP1.common.dedup.metrics &


picard MarkDuplicates \
    I=${sample}_HAP2.common.bam \
    O=${sample}_HAP2.common.dup_marked.bam \
    CREATE_INDEX=false \
    VALIDATION_STRINGENCY=SILENT \
    REMOVE_DUPLICATES=FALSE \
    M=${sample}_HAP2.common.dedup.metrics &


picard MarkDuplicates \
    I=${sample}_HAP1.unique.bam \
    O=${sample}_HAP1.unique.dup_marked.bam \
    CREATE_INDEX=false \
    VALIDATION_STRINGENCY=SILENT \
    REMOVE_DUPLICATES=FALSE \
    M=${sample}_HAP1.unique.dedup.metrics &


picard MarkDuplicates \
    I=${sample}_HAP2.unique.bam \
    O=${sample}_HAP2.unique.dup_marked.bam \
    CREATE_INDEX=false \
    VALIDATION_STRINGENCY=SILENT \
    REMOVE_DUPLICATES=FALSE \
    M=${sample}_HAP2.unique.dedup.metrics &

In [ ]:
# report the number of duplicate reads
echo HAP1_common.bam has `cat ${sample}_HAP1.common.dedup.metrics | awk 'NR==8' | cut -f2` reads, `cat ${sample}_HAP1.common.dedup.metrics | awk 'NR==8' | cut -f6` of which are duplicates
echo HAP2_common.bam has `cat ${sample}_HAP2.common.dedup.metrics | awk 'NR==8' | cut -f2` reads, `cat ${sample}_HAP2.common.dedup.metrics | awk 'NR==8' | cut -f6` of which are duplicates
echo HAP1_unique.bam has `cat ${sample}_HAP1.unique.dedup.metrics | awk 'NR==8' | cut -f2` reads, `cat ${sample}_HAP1.unique.dedup.metrics | awk 'NR==8' | cut -f6` of which are duplicates
echo HAP2_unique.bam has `cat ${sample}_HAP2.unique.dedup.metrics | awk 'NR==8' | cut -f2` reads, `cat ${sample}_HAP2.unique.dedup.metrics | awk 'NR==8' | cut -f6` of which are duplicates

In [ ]:
# liftover common ones to hg38 - check how many are mapping exactly at the same spot
bamToBed -i ${sample}_HAP1.common.bam > ${sample}_HAP1.common.bed &
bamToBed -i ${sample}_HAP2.common.bam > ${sample}_HAP2.common.bed &

wait

liftOver ${sample}_HAP1.common.bed ${sample}_HAP1_to_HG38.chain ${sample}_HAP1.common.hg38.bed unmapp1 &
liftOver ${sample}_HAP2.common.bed ${sample}_HAP2_to_HG38.chain ${sample}_HAP2.common.hg38.bed unmapp2 &

wait

In [ ]:
# report the number of reads that couldn't be lifted over
echo Out of `cat ${sample}_HAP1.common.bed | wc -l` HAP1.common reads `cat unmapp1 | grep -v '^#' | wc -l` could not be lifted over to hg38 >> postprocess_haplotype_bam_files.log
echo Out of `cat ${sample}_HAP2.common.bed | wc -l` HAP1.common reads `cat unmapp2 | grep -v '^#' | wc -l` could not be lifted over to hg38 >> postprocess_haplotype_bam_files.log

In [ ]:
# find the unambigously mapping reads
intersectBed -a ${sample}_HAP1.common.hg38.bed -b ${sample}_HAP2.common.hg38.bed -r -f 1 -wa > ${sample}_HAP1_and_HAP2_common_reads
cat ${sample}_HAP1_and_HAP2_common_reads | cut -f4 | sort -u > ${sample}_HAP1_and_HAP2_common_read.IDs

# report the number of unambigously mapping reads
echo `cat ${sample}_HAP1_and_HAP2_common_read.IDs | wc -l` reads are unambigously mapping between ${sample}_HAP1.common and ${sample}_HAP2.common >> postprocess_haplotype_bam_files.log

In [ ]:
# filter out reads mapped to different locations
picard FilterSamReads I=${sample}_HAP1.common.bam O=${sample}_HAP1.common.no_amb.bam FILTER=includeReadList READ_LIST_FILE=${sample}_HAP1_and_HAP2_common_read.IDs
picard FilterSamReads I=${sample}_HAP2.common.bam O=${sample}_HAP2.common.no_amb.bam FILTER=includeReadList READ_LIST_FILE=${sample}_HAP1_and_HAP2_common_read.IDs

In [ ]:
# filter out reads that are identified as duplicates (in either of the common.bam files)
samtools view ${sample}_HAP1.common.dup_marked.bam | awk '$2==1040 || $2==1024' | cut -f1 | sort -u > ${sample}_HAP1.common.dup_read_IDs
samtools view ${sample}_HAP2.common.dup_marked.bam | awk '$2==1040 || $2==1024' | cut -f1 | sort -u > ${sample}_HAP2.common.dup_read_IDs
cat ${sample}_HAP1.common.dup_read_IDs ${sample}_HAP2.common.dup_read_IDs | sort -u > ${sample}.common.dup_read_IDs


In [ ]:
# report the total number of duplicated reads
echo There are collectively `cat ${sample}.common.dup_read_IDs | wc -l` duplicate reads in ${sample}_HAP1.common and ${sample}_HAP2.common >> postprocess_haplotype_bam_files.log

In [ ]:
# filter out duplicated reads
picard FilterSamReads I=${sample}_HAP1.common.no_amb.bam O=${sample}_HAP1.common.no_amb.no_dup.bam FILTER=excludeReadList READ_LIST_FILE=${sample}.common.dup_read_IDs
picard FilterSamReads I=${sample}_HAP2.common.no_amb.bam O=${sample}_HAP2.common.no_amb.no_dup.bam FILTER=excludeReadList READ_LIST_FILE=${sample}.common.dup_read_IDs

In [ ]:
## Pileup over heterozygous sites
# extract necessay fields from WGS genome vcf file
java -Xmx100G -jar /data/leuven/software/biomed/SnpEff/4.3p/SnpSift.jar extractFields \
    ${wgs_folder}/${sample}/snpeff/${sample}_phased_variants.HQ.ann_dbNSFP3.snp150.COSMIC.vcf.gz \
    -s '|' \
    -e '.' CHROM POS ID REF ALT DP GEN[*].AD GEN[*].PS GEN[*].GT AF \
  > ${sample}_WGS_variants_with_extra_info

sed 1d ${sample}_WGS_variants_with_extra_info \
    | awk '{print $1"\t"$2-1"\t"$2"\t"$4"/"$5}' \
    | intersectBed -a stdin -b ${wgs_folder}/${sample}/AS_ATAC/${sample}_REF_HAP1_HAP2_peaks.merged_and_filtered.bed -wa \
  > ${sample}_HET_SNVs.hg38.bed

In [ ]:
# report the number of heterozygous sites that are considered for allele-specific activity

In [ ]:
echo Pileup over heterozygous sites >> postprocess_haplotype_bam_files.log
echo There are `cat ${sample}_HET_SNVs.hg38.bed | wc -l` heterozygous sites in the genome >> postprocess_haplotype_bam_files.log

In [ ]:
# liftover het sites to hap1 and hap2
liftOver ${sample}_HET_SNVs.hg38.bed ${sample}.refTOhap1.chain ${sample}_HET_SNVs.hap1.bed unmapp1
liftOver ${sample}_HET_SNVs.hg38.bed ${sample}.refTOhap2.chain ${sample}_HET_SNVs.hap2.bed unmapp2

In [ ]:
# filter out duplicate reads from HAP1 and HAP1 mapped reads
samtools view ${sample}_HAP1.unique.dup_marked.bam | awk '$2==1040 || $2==1024' | cut -f1 | sort -u > ${sample}_HAP1.unique.dup_read_IDs
samtools view ${sample}_HAP2.unique.dup_marked.bam | awk '$2==1040 || $2==1024' | cut -f1 | sort -u > ${sample}_HAP2.unique.dup_read_IDs

picard FilterSamReads I=${sample}_HAP1.unique.bam O=${sample}_HAP1.unique.no_dup.bam FILTER=excludeReadList READ_LIST_FILE=${sample}_HAP1.unique.dup_read_IDs
picard FilterSamReads I=${sample}_HAP2.unique.bam O=${sample}_HAP2.unique.no_dup.bam FILTER=excludeReadList READ_LIST_FILE=${sample}_HAP2.unique.dup_read_IDs

In [ ]:
# report the number of used reads for allele counting
echo Number of reads used for pileup: >> postprocess_haplotype_bam_files.log
echo ${sample}_HAP1.unique:`samtools view ${sample}_HAP1.unique.no_dup.bam | cut -f1 |sort -u | wc -l ` >> postprocess_haplotype_bam_files.log
echo ${sample}_HAP2.unique:`samtools view ${sample}_HAP2.unique.no_dup.bam | cut -f1 |sort -u | wc -l ` >> postprocess_haplotype_bam_files.log
echo ${sample}_HAP1.common:`samtools view ${sample}_HAP1.common.no_amb.no_dup.bam | cut -f1 |sort -u | wc -l ` >> postprocess_haplotype_bam_files.log
echo ${sample}_HAP2.common:`samtools view ${sample}_HAP2.common.no_amb.no_dup.bam | cut -f1 |sort -u | wc -l ` >> postprocess_haplotype_bam_files.log

In [ ]:
# pileup using unique reads
# hap1
samtools mpileup -Bf ${sample}.hap1.fa -l ${sample}_HET_SNVs.hap1.bed ${sample}_HAP1.unique.no_dup.bam > ${sample}_HET_SNVs.hap1_unique.pileup &
# hap2
samtools mpileup -Bf ${sample}.hap2.fa -l ${sample}_HET_SNVs.hap2.bed ${sample}_HAP2.unique.no_dup.bam > ${sample}_HET_SNVs.hap2_unique.pileup &

wait

# pileup using common reads
#hap1
samtools mpileup -Bf ${sample}.hap1.fa -l ${sample}_HET_SNVs.hap1.bed ${sample}_HAP1.common.no_amb.no_dup.bam > ${sample}_HET_SNVs.hap1_common.pileup &
# hap2
samtools mpileup -Bf ${sample}.hap2.fa -l ${sample}_HET_SNVs.hap2.bed ${sample}_HAP2.common.no_amb.no_dup.bam > ${sample}_HET_SNVs.hap2_common.pileup &

wait

# generate counts from pileup files
cat ${sample}_HET_SNVs.hap1_unique.pileup | ~/lcb/zkalender/software/pileup2bed_depth_vaf_with_allele_counts.awk > ${sample}_HET_SNVs.hap1_unique.counts.bed
cat ${sample}_HET_SNVs.hap2_unique.pileup | ~/lcb/zkalender/software/pileup2bed_depth_vaf_with_allele_counts.awk > ${sample}_HET_SNVs.hap2_unique.counts.bed

cat ${sample}_HET_SNVs.hap1_common.pileup | ~/lcb/zkalender/software/pileup2bed_depth_vaf_with_allele_counts.awk > ${sample}_HET_SNVs.hap1_common.counts.bed
cat ${sample}_HET_SNVs.hap2_common.pileup | ~/lcb/zkalender/software/pileup2bed_depth_vaf_with_allele_counts.awk > ${sample}_HET_SNVs.hap2_common.counts.bed

# liftover everything back to hg38
liftOver ${sample}_HET_SNVs.hap1_unique.counts.bed ${sample}_HAP1_to_HG38.chain ${sample}_HET_SNVs.hap1_unique.counts.hg38.bed unmapp1
liftOver ${sample}_HET_SNVs.hap1_common.counts.bed ${sample}_HAP1_to_HG38.chain ${sample}_HET_SNVs.hap1_common.counts.hg38.bed unmapp1

liftOver ${sample}_HET_SNVs.hap2_unique.counts.bed ${sample}_HAP2_to_HG38.chain ${sample}_HET_SNVs.hap2_unique.counts.hg38.bed unmapp1
liftOver ${sample}_HET_SNVs.hap2_common.counts.bed ${sample}_HAP2_to_HG38.chain ${sample}_HET_SNVs.hap2_common.counts.hg38.bed unmapp1

In [ ]:
# filter out INDELs and homozygous variants AND Overlap with ATAC peaks
sed 1d ${sample}_WGS_variants_with_extra_info \
  | awk -v 'OFS=\t' '{ if (length($4)==1 && length($5)==1) { if ($9=="0|1" || $9=="1|0") { print $1, $2-1, $2, $3, $4, $5, $6, $7, $9, $10 } } }' \
  | intersectBed \
        -a stdin \
        -b ${wgs_folder}/${sample}/AS_ATAC/${sample}_REF_HAP1_HAP2_peaks.merged_and_filtered.bed \
        -wo \
  | cut -f 1-15 \
  | sortBed -i stdin \
  | awk '{print $0"\t'${sample}'_SNV_"NR}' \
  > heterozygote_SNVs_in_peaks.bed

In [ ]:
cat heterozygote_SNVs_in_peaks.bed | cut -f 1-3,16 > heterozygote_SNVs_in_peaks.NAMED.bed

echo Generating counts >> postprocess_haplotype_bam_files.log

# intersect the named bed file with unique files
awk '$4!=0' ${sample}_HET_SNVs.hap1_unique.counts.hg38.bed | intersectBed -a heterozygote_SNVs_in_peaks.NAMED.bed -b stdin -wo | cut -f 4,8 > HET_SNVs_hap1_counts
awk '$4!=0' ${sample}_HET_SNVs.hap2_unique.counts.hg38.bed | intersectBed -a heterozygote_SNVs_in_peaks.NAMED.bed -b stdin -wo | cut -f 4,8 > HET_SNVs_hap2_counts

# intersect the named bed file with common counts
awk '$4!=0' ${sample}_HET_SNVs.hap1_common.counts.hg38.bed | intersectBed -a stdin -b heterozygote_SNVs_in_peaks.NAMED.bed -wo | cut -f 4,8 > HET_SNVs_hap1_common_counts
awk '$4!=0' ${sample}_HET_SNVs.hap2_common.counts.hg38.bed | intersectBed -a stdin -b heterozygote_SNVs_in_peaks.NAMED.bed -wo | cut -f 4,8 > HET_SNVs_hap2_common_counts

echo Done >> postprocess_haplotype_bam_files.log